In [5]:
import pandas as pd
from pathlib import Path
from pandas.errors import EmptyDataError

def summarize_scenario(run_dir: Path, scenario: str) -> dict:
    # decision log은 항상 존재
    dec = pd.read_csv(run_dir / scenario / "decision_log.csv")

    start_ab = dec.iloc[0]["allocation_id"]
    final_ab = dec.iloc[-1]["allocation_id"]

    fb_path = run_dir / scenario / "fallback_log.csv"

    # fallback 기본값 (cold 대응)
    fallback_steps = []
    fallback_path = None
    fallback_count = 0

    if fb_path.exists():
        try:
            fb = pd.read_csv(fb_path)

            if len(fb) > 0:
                fallback_steps = fb["step"].tolist()
                fallback_path = (
                    fb[["previous_allocation", "fallback_allocation"]]
                    .astype(str)
                    .apply(lambda r: f"{r[0]}→{r[1]}", axis=1)
                    .tolist()
                )
                fallback_path = " → ".join(fallback_path)
                fallback_count = len(fallback_steps)

        except EmptyDataError:
            # cold처럼 fallback 자체가 없는 경우
            pass

    return {
        "scenario": scenario,
        "start_antibody": start_ab,
        "final_antibody": final_ab,
        "fallback_count": fallback_count,
        "fallback_steps": fallback_steps,
        "fallback_path": fallback_path,
    }

In [4]:
RUN_DIR = Path("../artifacts/core11/core11_demo_001")

summary = []
for scenario in ["cold", "hot", "oscillation"]:
    summary.append(summarize_scenario(RUN_DIR, scenario))

summary_df = pd.DataFrame(summary)
summary_df

EmptyDataError: No columns to parse from file